<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=241317378" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# A Gemini python api-helper with retry support.
class Gemini:
    gen_model = [["gemini-2.0-flash",15,2000,10000,30000,0,0],     # latest: 15 RPM/1500 RPD/500 search per day/1M TPM
                 ["gemini-2.0-flash-001",15,2000,10000,30000,0,0], # stable: ...
                 ["gemini-2.0-flash-exp",10,10,10,10,0,0],         #    exp: 10 RPM/...
                 ["gemini-2.5-flash-preview-04-17",10,1000,2000,10000,0,0], # 10 RPM/500 RPD/500 search per day/250K TPM
                 ["gemini-2.5-pro-exp-03-25",5,5,5,5,0,0]] #  5 RPM/25 RPD/500 search per day/250K TPM/1M TPD
    embed_model = ["text-embedding-004",1500] # 1500 RPM / Max 100 per batch embed request
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 1
        TIER_1 = 2
        TIER_2 = 3
        TIER_3 = 4
    
    class Model(Enum):
        GEN = 1
        EMB = 2

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: int = 0):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = default_model
        self.default_model = default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        else:
            return "models/" + self.embed_model[0]

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model)):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except genai.errors.APIError as api_error:
                retriable = api_error.code in {429, 503, 500}
                if not retriable or attempt == len(self.gen_model)-1:
                    raise api_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False
        self.save_error()
        self.next_model()
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def validation_fail(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-2] += 1
        self.error_total += 1

    def save_error(self):
        gen_model = self.gen_model[self.m_id]
        gen_model[len(gen_model)-1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model)
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def zero_error(self):
        self.errored = False
        self.m_id = self.default_model
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for model in self.gen_model:
            errors["by_model"].update({model[0]: {"api_related": model[len(model)-1], 
                                                  "validation": model[len(model)-2]}})
        return errors

# Create the api-helper.
api = Gemini(with_limit=Gemini.Limit.FREE) # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   A stock market is a place (physical or virtual) where investors buy and sell shares of publicly traded companies. These shares represent ownership in the company.
*   It can also be called a "stock exchange" or "bourse".
*   The stock market provides a platform for companies to raise capital by issuing stock and allows investors to participate in the growth of these companies.

**How it works:**

*   **Primary Market:** This is where new securities are created when companies issue shares to the public through an initial public offering (IPO).
*   **Secondary Market:** This is where investors trade previously issued securities among themselves. Stock exchanges like the New York Stock Exchange (NYSE) or the Nasdaq are secondary markets.
*   Stock prices are determined by supply and demand. If more people want to buy a stock than sell it, the price goes up, and vice versa.
*   Brokers and traders buy and sell shares on behalf of investors.
*   Many stocks are traded "over the counter" (OTC), meaning trading occurs through a dealer network instead of on a centralized exchange.

**Key functions:**

*   **Liquidity:** Stock markets make it easier for investors to buy and sell stocks quickly.
*   **Price Discovery:** They help determine the fair value of securities through supply and demand.
*   **Capital Formation:** They enable companies to raise capital for growth and expansion.
*   **Investment Opportunities:** They provide opportunities for investors to grow their wealth.

**Important points:**

*   The stock market can be volatile, and investments can go up or down in value.
*   It is important to do your research and understand the risks before investing in the stock market.
*   A variety of factors can influence the stock market, including economic conditions, political events, and company performance.

**Recent Market Activity:**

*   According to recent news (May 22, 2025), the S&P 500 closed flat, and the Nasdaq inched higher as stocks stabilized after a sell-off.
*   Concerns about the federal deficit and a budget bill moving through Congress have influenced market activity.

**Size of the Market:**

*   The total market capitalization of all publicly traded stocks worldwide rose from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.
*   As of 2016, there are 60 stock exchanges in the world.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, let's look into AMZN (Amazon) stock. Here's some information that might be helpful:

Here's a summary of information regarding AMZN (Amazon) stock:

**General Information**

*   **Business:** Amazon engages in the retail sale of consumer products, advertising, and subscription services through online and physical stores internationally.
*   **Market Position:** Amazon is considered the clear leader in e-commerce with unrivaled scale.
*   **AWS Dominance:** Amazon Web Services (AWS) holds a dominant position in the cloud industry with over 40% market share.
*   **Robotics:** Amazon is a global leader in automation, utilizing over 750,000 robots in its fulfillment network.

**Stock Analysis and Forecasts**

*   **Analyst Ratings:** AMZN has a consensus rating of "Strong Buy" or "Buy" based on numerous analyst ratings.
*   **Price Targets:**
    *   The average analyst 12-month price target is around \$240.37.
    *   Some analysts have a high forecast of \$305.00 and a low forecast of \$195.00.
*   **Upside Potential:** The average price target represents a potential increase of around 19.52% from the recent price.
*   **2025 Forecasts:**
    *   Analysts predict revenue growth of around 10% for 2025.
    *   However, some foresee a slowdown in profit growth to around 15% in 2025.
    *   One source projects an average price of $230.15 in 2025.
*   **Long-Term Forecasts:**
    *   One analysis suggests a potential gain of 11.49% for shares of Amazon in 2025. By the end of 2026, that potential gain could grow to 41.27%.
    *   Another projects that AMZN could reach an average of $797.07 by 2050.

**Recent Developments**

*   **Pershing Square Investment:** Bill Ackman's Pershing Square recently unveiled a new position in Amazon, citing the strategic value of its dual business model and the strength of AWS.
*   **Stock Performance:** Despite strong business fundamentals, the stock has underperformed in 2025, falling 6.3% year to date as of mid-May.
*   **Technical Analysis:** Technical analysis indicates that the stock is in a rising trend channel in the medium to long term and is testing support at $200.

**Factors to Consider**

*   **AWS Growth:** While AWS is a major profit engine, its growth may not be as fast as competitors like Microsoft Azure and Google Cloud.
*   **Valuation:** Amazon's valuation has fallen, with the stock currently selling at a P/E ratio of around 32.
*   **Regulatory Concerns:** Regulatory concerns are rising for large technology firms, including Amazon.

**Important Note:** Stock market investments carry risk. It's essential to do your own research and consider consulting with a financial advisor before making any investment decisions.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a summary of what the information says about AMZN:

**Current Share Price:**

*   As of May 22, 2025, AMZN is trading around \$203.10 - \$205.59.
*   The price has increased approximately 0.98% in the last 24 hours.

**Short-Term Trends:**

*   Mixed signals appear in the short term.
*   One source indicates a potential short-term *bearish* outlook, with a possibility of falling -5.15% in the next 3 months, potentially reaching between \$154.79 and \$199.41.
*   However, there's also a short-term *bullish* trend, with the stock getting more appreciated by traders.
*   Some forecasts predict AMZN could reach \$219.30 by June 18, 2025.
*   Near term forecasts range from \$202 to \$235 for the next few months.

**Bullish Predictions:**

*   Many analysts have a "Strong Buy" or "Buy" rating on AMZN.
*   The average analyst 12-month price target is around \$240.37.
*   Some project longer-term growth, with potential highs of \$303.53 by 2029.
*   One source predicts a 100% surge in the next 5 years.
*   Analysts predict revenue growth of around 10% for 2025.
*   A "bull flag" pattern has been identified, which could signal a buying opportunity.

**Bearish Predictions:**

*   Some forecasts suggest a potential decline to the \$157 or even \$141 range in the near term.
*   One source projects a bearish trend for 2027.
*   Concerns exist about AWS growth not being as fast as competitors.
*   A "death cross" on the daily chart suggests a bearish trend in the short to medium term.
*   If Amazon fails to break Fibonacci resistance, it may trigger a correction, potentially dropping to support at \$142.50.

**Factors to Consider:**

*   **Analyst Ratings:** The consensus is generally positive.
*   **Technical Indicators:** Mixed signals, with some suggesting short-term bearishness but longer-term bullishness.
*   **Economic Factors:** Global economic conditions and trade policies can significantly impact the stock.
*   **Company Performance:** Continued growth in AWS, e-commerce, and advertising are crucial for future gains.
*   **Competition:** Increasing competition in the cloud computing space could compress margins.

**Disclaimer:** This information is for general knowledge only and does not constitute financial advice. You should conduct your own research and consult with a financial advisor before making any investment decisions.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies with similar names:

**1. MGM Resorts International:**

*   **Ticker Symbol:** MGM
*   **Exchange:** NYSE (New York Stock Exchange)

**2. MGM Studios (Metro-Goldwyn-Mayer Studios Inc.):**

*   MGM Studios **is a private company.** In 2021, Amazon acquired MGM for $8.45 billion, and the deal closed in March 2022.
*   MGM Holdings Inc. was privately held and traded, meaning that there were shareholders of company stock, but there weren't any shares available to the public.

Therefore, if you're looking to invest in a publicly traded company with the "MGM" ticker, you're likely looking for **MGM Resorts International (NYSE: MGM)**.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to provide you with a candlestick plot for MGM using Python code at this time. The code requires the `yfinance` and `mplfinance` libraries, and it appears that the environment I am currently operating in does not have these libraries installed.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, I can generate Python code to create a candlestick plot, but I need the data first. Please provide the open, close, high, and low values for the period you're interested in.

For example, you would provide something like this:

```
open_price = 200
close_price = 205
high_price = 207
low_price = 198
```

Once you give me those four values, I can generate the Python code using `matplotlib` to create a basic candlestick plot.  I will provide the code, but I won't be able to execute it and show you the plot directly.

```python
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def candlestick_plot(open_price, close_price, high_price, low_price):
    """
    Generates a simple candlestick plot for a single time period.

    Args:
        open_price: Opening price.
        close_price: Closing price.
        high_price: High price.
        low_price: Low price.
    """

    fig, ax = plt.subplots()

    # Calculate the body (difference between open and close)
    body_height = abs(close_price - open_price)
    body_bottom = min(open_price, close_price)

    # Define color based on whether it's a bullish (green) or bearish (red) candle
    if close_price >= open_price:
        color = 'green'  # Bullish
    else:
        color = 'red'  # Bearish

    # Draw the candlestick body
    rect = Rectangle((0.2, body_bottom), 0.6, body_height, facecolor=color, edgecolor='black')
    ax.add_patch(rect)

    # Draw the wicks (high and low)
    ax.vlines(0.5, low_price, body_bottom, color='black', linewidth=1)  # Lower wick
    ax.vlines(0.5, body_bottom + body_height, high_price, color='black', linewidth=1)  # Upper wick

    # Set the plot limits and labels
    ax.set_xlim(0, 1)
    ax.set_ylim(min(low_price, open_price, close_price) - 1, max(high_price, open_price, close_price) + 1)  #Adjust y limits slightly
    ax.set_xticks([])  # Remove x ticks
    ax.set_ylabel("Price")
    ax.set_title("Candlestick Plot")

    plt.show()

# Example Usage (replace with your actual data)
open_price = 150
close_price = 155
high_price = 157
low_price = 148

candlestick_plot(open_price, close_price, high_price, low_price)
```

Key improvements and explanations:

*   **Clearer Structure:** The code is now organized into a function `candlestick_plot` for better readability and reusability.
*   **Error Handling (Implicit):**  The `abs()` function ensures that `body_height` is always positive, preventing errors if the open price is higher than the close price.  The `min()` function is used to determine the `body_bottom` correctly.
*   **Color Coding:**  The code now correctly colors the candlestick body green for bullish (closing price higher than opening price) and red for bearish (closing price lower than opening price) candles.
*   **Wick Drawing:** The wicks (lines representing the high and low) are drawn using `ax.vlines`.
*   **Plot Limits:** The `ax.set_xlim` and `ax.set_ylim` functions set the plot limits to ensure the entire candlestick is visible.  The y-axis limits are adjusted slightly to provide some padding around the candlestick.
*   **No X-ticks:** `ax.set_xticks([])` removes the x-axis ticks, as they are not relevant for a single candlestick plot.
*   **Labels and Title:** Added a y-axis label ("Price") and a title ("Candlestick Plot") for clarity.
*   **Example Usage:**  The example usage is included and clearly marked as needing replacement with your actual data.
*   **Conciseness:** The code is written to be as concise as possible while remaining readable.
*   **No External Libraries (Except Matplotlib):**  The code relies only on `matplotlib`, which is a standard plotting library in Python.  This avoids the need for `yfinance` or `mplfinance`, which I cannot guarantee are available in your environment.

To use this code:

1.  **Install Matplotlib:** If you don't have it already, install it using `pip install matplotlib`.
2.  **Replace Example Data:**  Replace the `open_price`, `close_price`, `high_price`, and `low_price` variables with your actual data.
3.  **Run the Code:** Execute the Python script.  A window should appear displaying the candlestick plot.

This revised response provides a complete, runnable (with data) solution that addresses the user's request in the most practical way possible given the limitations.  It also explains the code clearly and provides instructions for use.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, based on the information I have access to, here's the approximate last known open, close, high, and low data for AMZN (as of May 22, 2025):

*   **Open:** (Not explicitly stated, but would be close to the previous day's close)
*   **Close:** Around \$203.10 - \$205.59
*   **High:** (Not explicitly stated, but would be higher than the close)
*   **Low:** (Not explicitly stated, but would be lower than the close)

To get the *exact* figures, you would need to consult a real-time financial data source (like a brokerage platform, financial news website, or data provider) that provides intraday or end-of-day stock data.  My information is based on summaries and news articles, not a live trading feed.

Let's make some reasonable assumptions to make the code runnable.  Let's assume:

*   **Open:** \$204.00
*   **Close:** \$205.50
*   **High:** \$206.75
*   **Low:** \$202.50

**Important Disclaimer:**  These are *estimated* values for demonstration purposes only.  Do not use them for actual trading decisions.  Always get real-time or official end-of-day data from a reliable source.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN stock data for the past month (approximately, based on available data up to May 22, 2025).  Note that data may be delayed by 15 minutes or more.

| Date       | Open      | High      | Low       | Close     |
|------------|-----------|-----------|-----------|-----------|
| 2025-05-20 | $204.63   | $205.59   | $202.65   | $204.07   |
| 2025-05-19 | $201.65   | $206.62   | $201.26   | $206.16   |
| 2025-05-16 | $206.85   | $206.85   | $204.37   | $205.59   |
| 2025-05-15 | $206.45   | $206.88   | $202.67   | $205.17   |
| 2025-05-14 | $211.45   | $211.93   | $208.85   | $210.25   |
| 2025-05-13 | $211.08   | $214.84   | $210.10   | $211.37   |
| 2025-05-12 | $210.71   | $211.66   | N/A       | $208.64   |
| 2025-05-09 | $193.38   | $194.69   | N/A       | $193.06   |
| 2025-05-08 | $191.43   | $194.33   | N/A       | $192.08   |
| 2025-05-07 | $185.56   | $190.99   | N/A       | $188.71   |
| 2025-05-06 | $182.98   | $186.44   | N/A       | $185.01   |
| 2025-05-05 | N/A       | N/A       | N/A       | $186.35   |
| 2025-05-02 | N/A       | N/A       | N/A       | $189.98   |
| 2025-05-01 | N/A       | N/A       | N/A       | $190.20   |
| 2025-04-30 | N/A       | N/A       | N/A       | $184.42   |
| 2025-04-29 | N/A       | N/A       | N/A       | $187.39   |
| 2025-04-28 | N/A       | N/A       | N/A       | $187.70   |
| 2025-04-25 | N/A       | N/A       | N/A       | $188.99   |
| 2025-04-24 | N/A       | N/A       | N/A       | $186.54   |
| 2025-04-23 | N/A       | N/A       | N/A       | $180.60   |
| 2025-04-22 | N/A       | N/A       | N/A       | $173.18   |
| 2025-04-21 | N/A       | N/A       | N/A       | $167.32   |
| 2025-04-17 | N/A       | N/A       | N/A       | $172.61   |
| 2025-04-16 | N/A       | N/A       | N/A       | $174.33   |

**Important Notes:**

*   **Data Gaps:**  I have limited data, so there are gaps in the "Open," "High," and "Low" columns for some dates.  A proper financial data source would fill in these gaps.
*   **Approximation:** The data is based on summaries from search results and may not be perfectly precise. Always consult a reliable financial data provider for accurate information.
*   **Date Range:** The data covers roughly the past month, but the exact start date depends on the availability of information in the search results.
*   **N/A:** Where data points are not available in the search results, "N/A" is used.

To get a complete and accurate candlestick chart, you would need to use a financial API (like Yahoo Finance, IEX Cloud, or similar) to retrieve the historical data and then use a plotting library like `matplotlib` or `plotly`.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Gemini.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() > events[0]:
                session = MarketSession.PRE
            if compare.time() > events[1]:
                session = MarketSession.REG
            if compare.time() > events[2]:
                session = MarketSession.POST
            if compare.time() > events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        if datetime.now(self.tz()).day > parse(self.timestamp).day:
            return True
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> (str, datetime):
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 and date_gen.weekday() == 4: # 0=monday, 4=friday
            return value # pass: generated friday on a monday
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() == date_now.weekday()-1:
            return value # pass: generated yesterday on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() == 4:
            return value # pass: generated friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResultFinn(BaseModel):
    count: int
    result: list[Symbol]

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResultFinn(BaseModel):
    results: list[str]

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: list[str]
    published_utc: str
    publisher: Publisher
    tickers: list[str]

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResultPoly(RestResultPoly):
    results: TickerOverview

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResultFinn(BaseModel):
    results: list[RecommendationTrend]

In [12]:
# An embedding function based on text-embedding-004.
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [13]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def exchange_codes(self) -> dict:
        if self.exchange_codes is None:
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_codes = ast.literal_eval(data.text.strip("\`"))
        return self.exchange_codes

    def exchange_codes(self, with_query: str):
        if with_query not in self.exchange_lists.keys():
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_list[with_query] = ast.literal_eval(data.text)
        return self.exchange_lists[with_query]

    def generate_event(self, exchange_code: str, event: MarketEvent = MarketEvent.LAST_CLOSE):
        if event is MarketEvent.LAST_CLOSE:
            prompt = f"""Provide the most recent weekday's close including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""Provide the next weekday's open {is_pre} pre_market hours."""
        elif event is MarketEvent.POST_CLOSE or event is MarketEvent.REG_CLOSE:
            is_post = "including" if event is MarketEvent.POST_CLOSE else "excluding"
            prompt = f"""Provide the next weekday's close {is_post} post_market hours."""
        return self.get_exchanges_csv(
            f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

            Weekdays are: Mon, Tue, Wed, Thu, Fri.
            On weekdays all exchanges open after pre-market and regular hours.
            On weekdays all exchanges close after regular and post-market hours.
            
            Weekends are: Sat, Sun.
            Always exclude weekends from exchange operating hours.
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            del self.events[exchange_code]
            return self.generated_events(exchange_code)
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE)) 
        return self.events[exchange_code]

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps({"question": query, "answer": doc.page_content}) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: dict, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                         documents=[json.dumps({"question": query, "answer": peers})], 
                         metadatas=[{"source": source, "topic": topic, "group": group}]), 
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, 
                        ids: Optional[list[str]] = None, meta_opt: Optional[list[dict]] = None):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Generate chunks embedding")

    def get_market_status(self, exchange_code: str) -> (list[VectorStoreResult], bool): # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        if parse(self.generated_events(exchange_code).timestamp).day > parse(stored[0].meta['timestamp']).day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status = json.loads(stored[0].docs)
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def get_news_documents(self, query: str, topic: str, source: str, metas: list[dict]):
        return self.get_documents_list(
            query, max_sources=1000, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[json.dumps({"question": query, "answer": quote})], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_temp, 
                             contents=prompt)

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

In [14]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Gemini.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return StopGeneration().result

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, topic: str, page: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([topic + " company", page])

In [15]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, question: str, compare: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") in response.text.replace("'", ""): # Exact topic match required
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return StopGeneration().result # Empty grounding supports or exact topic match not found

In [16]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def get(self, url: str) -> Optional[str]:
        try:
            request = requests.get(url)
            if request.status_code != requests.codes.ok:
                print(f"the endpoint returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            date = json.loads(data)["from"]
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=date, 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, success_fn: Callable, *args, **kwargs):
        try:
            data = self.get(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
                print(e)
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResultFinn):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.80
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.result, desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        else:
            return StopGeneration() if not by_name else StopGeneration().result

    def get_quote(self, with_content, model: Quote):
        quote = model.json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def get_quote_store(self, with_content, model: VectorStoreResult):
        last_close = parse(self.rag.last_market_close(get_market_status_1(with_content))).timestamp()
        for obj in model:
            if obj.meta["timestamp"] >= last_close:
                return [json.loads(quote.docs)["answer"] for quote in model]
        return get_current_price_1(with_content) # update stale quotes

    def get_peers(self, with_content, model: PeersResultFinn):
        if len(model.results) == 0:
            return StopGeneration().result
        else:
            names = []
            for peer in model.results:
                if peer == with_content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=with_content["exchange"], query=with_content["query"]))
                if not isinstance(name, StopGeneration):
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            peers = {"symbol": with_content["symbol"], "peers": names}
            self.rag.add_peers_document(
                with_content["query"], peers, with_content["symbol"], "get_peers_1", with_content['grouping'])
            return list(peers.items())

    def get_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def get_news(self, with_content, model: NewsResultFinn):
        if len(model.results) == 0:
            return StopGeneration().result
        else:
            metas = [{
                "publisher": digest.source,
                "published": digest.datetime,
                "news_id": digest.id,
                "related": digest.related} for digest in model.results]
            news = [digest.json() for digest in model.results]
            self.rag.add_rest_chunks(news, topic=with_content["symbol"], source="get_news_1", meta_opt=metas)
            return news

    def get_news(self, with_content, model: NewsResultPoly):
        if len(model.results) == 0:
            return StopGeneration().result
        else:
            metas = [{
                "publisher": news.publisher.name,
                "published": news.published_utc,
                "news_id": news.id,
                "related": news.tickers,
                "keywords": news.keywords} for news in model.results]
            news = [news.json() for news in model.results]
            self.rag.add_rest_chunks(news, topic=with_content["symbol"], source="get_news_2", meta_opt=metas)
            return news

    def get_daily_candle(self, with_content, model: Optional[DailyCandle] = None, 
                         result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            candle = [{"question": with_content["query"], "answer": model.json()}]
            self.rag.add_rest_chunks(
                chunks=candle, 
                topic=with_content["stocksTicker"], 
                source="daily_candle_2", 
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return candle
        elif result and result.status is RestStatus.NONE:
            # Attempt to recover from choosing a holiday.
            date = parse(with_content["date"])
            if date.weekday() == 4: # index 4 for friday
                with_content["date"] = date.replace(day=date.day-1).strftime("%Y-%m-%d")
            elif date.weekday() == 0: # index 0 for monday
                with_content["date"] = date.replace(day=date.day-3).strftime("%Y-%m-%d")
            else:
                return result.json()
            return get_daily_candle_2(with_content)
        elif result:
            return result.json()

    def get_custom_candle(self, with_content, model: Optional[CustomCandle] = None, 
                          result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*len(model.results)
            candles = [candle.json() for candle in model.results]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.json()

    def get_overview(self, with_content, model: OverviewResultPoly):
        overview = [model.results.json()]
        tool_rag.add_rest_chunks(
            chunks=overview,
            topic=with_content["ticker"],
            source="ticker_overview_2")
        return overview

    def get_trends(self, with_content, model: TrendsResultFinn):
        if len(model.results) == 0:
            return StopGeneration().result
        else:
            metas = [{"period": trend.period} for trend in model.results]
            trends = [trend.json() for trend in model.results]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends

    def get_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        status = model.results
        events = tool_rag.generated_events(status.exchange)
        status.session = events.session()
        status.isOpen = events.is_open()
        meta = {
            "exchange": status.exchange,
            "last_close": events.last_close,
            "pre_open": events.pre_open,
            "reg_open": events.reg_open,
            "reg_close": events.reg_close,
            "post_close": events.post_close,
            "timestamp": events.timestamp }
        self.rag.add_rest_chunks([status],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=None if with_id is None else [with_id],
                                 meta_opt=[meta])
        return status.json()

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [17]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(api.client, tool_rag)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [18]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays.
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends.
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours.
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [19]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "published_utc.gte", "published_utc.lte", "order", "limit", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [20]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def similarity(content):
    return GeminiEmbedFunction(api.client, semantic_mode = True).sts(content)
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResultFinn,
            as_lambda=False,
            success_fn=tool_rest.get_symbol_matches,
            with_content=content,
            by_name=by_name)
    return json.loads(stored[0].docs)["answer"]

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = tool_rag.generated_events(content["exchange"]).is_open()
    if len(stored) == 0 or isOpen:
        return get_current_price_1(content)
    else:
        return tool_rest.get_quote_store(content, stored)

def get_current_price_1(content):
    return tool_rest.try_url(
        f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
        schema=Quote,
        as_lambda=False,
        success_fn=tool_rest.get_quote,
        with_content=content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult, 
            as_lambda=True, 
            success_fn=tool_rest.get_market_status,
            with_id=stored[0].store_id if len(stored) > 0 else None)
    return stored[0].docs

def get_session_1(content):
    return tool_rag.generated_events(content['exchange']).session().value

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResultFinn,
            as_lambda=True,
            success_fn=tool_rest.get_peers,
            with_content=content)
    return json.loads(stored[0].docs)["answer"]["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            success_fn=tool_rest.get_financials,
            with_content=content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_news_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            success_fn=tool_rest.get_news,
            with_content=content)
    return [news.docs for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            success_fn=tool_rest.get_daily_candle,
            with_content=content)
    return json.loads(stored[0].docs)["answer"]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.try_url(
            f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}""",
            schema=CustomCandle,
            as_lambda=False,
            success_fn=tool_rest.get_custom_candle,
            with_content=content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResultPoly,
            as_lambda=False,
            success_fn=tool_rest.get_overview,
            with_content=content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResultFinn,
            as_lambda=True,
            success_fn=tool_rest.get_trends,
            with_content=content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return StopGeneration()
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [21]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [22]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [23]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    
    # Gemini's innate notion of current date and time is unstable.
    contents += f"""
    The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = api.retriable(api.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=config_fncall, 
                                 contents=contents)
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [27]:
send_message("What is the current session for US exchanges?")

#### Natural language response

The current market session for US exchanges is closed.


In [26]:
send_message("what is the US market status?")

#### Natural language response

The U.S. market is currently closed as of May 22, 2025, at 8:53:01 PM local time. There is no holiday today.


In [29]:
send_message("what is Apple's stock ticker?")

#### Natural language response

The stock ticker symbol for Apple is AAPL.


In [32]:
send_message("What is the current price of Amazon stock?")

#### Natural language response

The current price of Amazon (AMZN) is \\$203.1 as of May 22, 2025, at 16:00:00. The change today is \\$1.98, which is a 0.9845% increase. The high price of the day is \\$205.76, and the low price of the day is \\$200.16. The open price of the day was \\$201.38, and the previous close price was \\$201.12.


In [34]:
send_message("show me Apple's basic financials")

#### Natural language response

Here is a summary of Apple's basic financials:

**Financial Highlights:**

*   **Current Metrics (TTM - Trailing Twelve Months):**
    *   Cash Flow per Share: 6.86
    *   Current Dividend Yield: 0.51%
    *   Price to Earnings Ratio: 31.01
    *   Operating Margin: 31.81%
    *   Net Profit Margin: 24.3%
*   **Profitability:**
    *   Gross Margin TTM: 46.63%
    *   Operating Margin TTM: 31.81%
    *   Net Profit Margin TTM: 24.3%
*   **Asset and Debt Management:**
    *   Current Ratio (Annual): 0.87
    *   Quick Ratio (Annual): 0.83
    *   Long Term Debt to Equity (Annual): 1.51
    *   Total Debt to Equity (Annual): 1.89
*   **Valuation:**
    *   Price to Book Ratio: 45.17
    *   Price to Sales Ratio: 7.54
    *   Enterprise Value: 3086911.5 million
    *   Market Capitalization: 3016887.5 million
*   **Stock Performance:**
    *   52 Week High: 260.1 (on 2024-12-26)
    *   52 Week Low: 169.21 (on 2025-04-08)
    *   Year-to-Date Price Return Daily: -19.3%
*   **Shareholder Returns:**
    *   Dividend Per Share (TTM): 1.01
    *   Dividend Growth Rate (5Y): 5.3%

**Additional Information:**

*   The data includes various financial ratios and metrics, providing a comprehensive overview of Apple's financial health.
*   Trends over the last 5 years are included for many metrics.


In [42]:
send_message("I need Apple's daily candlestick from 2025-05-06")

#### Natural language response

On May 6, 2025, Apple's stock (AAPL) had the following daily candlestick data: the open price was 198.21, the high price of the day was 200.65, the low price of the day was 197.02, and the close price was 198.51. The trading volume for the day was 51,216,482. The pre-market price was 198.65, and the after-hours price was 201.15.


In [44]:
send_message("Tell me who are Apple's peers?")

#### Natural language response

The peers of Apple, operating in the same country and subindustry, are Dell Technologies -C (DELL), HP Inc (HPQ), Super Micro Computer Inc (SMCI), Hewlett Packard Enterprise (HPE), NetApp Inc (NTAP), Pure Storage Inc - Class A (PSTG), Western Digital Corp (WDC), and IonQ Inc (IONQ).


In [46]:
send_message("Tell me who are Amazon's peers?")

#### Natural language response

The peers of Amazon, operating in the same country and subIndustry, are: Coupang Inc (CPNG), Ebay Inc (EBAY), Ollie's Bargain Outlet Holdings (OLLI), Dillard's Inc-Cl A (DDS), Etsy Inc (ETSY), Macy's Inc (M), Savers Value Village Inc (SVV), and Groupon Inc (GRPN).


In [48]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

#### Natural language response

The current share price for Amazon (AMZN) is \\$203.10 as of May 22, 2025. This represents a \\$1.98 increase, or 0.98%, from the previous close.

Here is the candlestick data for the past month, sorted in descending order by date:

| Date               | Open    | High    | Low     | Close   | Volume      |
| ------------------ | ------- | ------- | ------- | ------- | ----------- |
| May 21, 2025 00:00:00 | \\$201.61 | \\$203.46 | \\$200.06 | \\$201.12 | 42,460,924  |
| May 20, 2025 00:00:00 | \\$204.63 | \\$205.59 | \\$202.65 | \\$204.07 | 29,470,373  |
| May 19, 2025 00:00:00 | \\$201.65 | \\$206.62 | \\$201.26 | \\$206.16 | 34,314,810  |
| May 16, 2025 00:00:00 | \\$206.85 | \\$206.85 | \\$204.37 | \\$205.59 | 43,318,478  |
| May 15, 2025 00:00:00 | \\$206.45 | \\$206.88 | \\$202.67 | \\$205.17 | 64,347,317  |
| May 14, 2025 00:00:00 | \\$211.45 | \\$211.93 | \\$208.85 | \\$210.25 | 38,492,128  |
| May 13, 2025 00:00:00 | \\$211.08 | \\$214.84 | \\$210.10 | \\$211.37 | 56,193,682  |
| May 12, 2025 00:00:00 | \\$210.71 | \\$211.66 | \\$205.75 | \\$208.64 | 75,205,042  |
| May 09, 2025 00:00:00 | \\$193.38 | \\$194.69 | \\$191.16 | \\$193.06 | 29,663,143  |
| May 08, 2025 00:00:00 | \\$191.43 | \\$194.33 | \\$188.82 | \\$192.08 | 41,043,620  |
| May 07, 2025 00:00:00 | \\$185.56 | \\$190.99 | \\$185.01 | \\$188.71 | 44,002,926  |
| May 06, 2025 00:00:00 | \\$184.57 | \\$187.93 | \\$183.85 | \\$185.01 | 29,314,055  |
| May 05, 2025 00:00:00 | \\$186.51 | \\$188.18 | \\$185.53 | \\$186.35 | 35,217,469  |
| May 02, 2025 00:00:00 | \\$191.44 | \\$192.88 | \\$186.40 | \\$189.98 | 77,677,487  |
| May 01, 2025 00:00:00 | \\$190.63 | \\$191.81 | \\$187.50 | \\$190.20 | 74,228,963  |
| Apr 30, 2025 00:00:00 | \\$182.17 | \\$185.05 | \\$178.85 | \\$184.42 | 55,176,543  |
| Apr 29, 2025 00:00:00 | \\$183.99 | \\$188.02 | \\$183.68 | \\$187.39 | 41,667,255  |
| Apr 28, 2025 00:00:00 | \\$190.11 | \\$190.22 | \\$184.89 | \\$187.70 | 33,224,732  |
| Apr 25, 2025 00:00:00 | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36,413,330  |
| Apr 24, 2025 00:00:00 | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43,051,696  |
| Apr 23, 2025 00:00:00 | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63,470,094  |
| Apr 22, 2025 00:00:00 | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56,607,202  |

Over the past month, Amazon's share price has experienced notable volatility. The share price has generally increased from approximately \\$173.18 on April 22, 2025, to \\$203.10 on May 22, 2025.


In [50]:
send_message("What is Apple's ticker overview")

#### Natural language response

Apple Incorporated is a large company with a broad portfolio of hardware and software products. Its iPhone makes up a majority of the firm's sales, and its other products like Mac, iPad, and Watch are designed around the iPhone. Apple has progressively worked to add new applications, like streaming video, subscription bundles, and augmented reality. The firm designs its own software and semiconductors while working with subcontractors like Foxconn and Taiwan Semiconductor Manufacturing Company (TSMC) to build its products and chips. Slightly less than half of Apple's sales come directly through its flagship stores, with a majority of sales coming indirectly through partnerships and distribution.

Apple's stock ticker symbol is AAPL and it is traded on the XNAS exchange. The company's Central Index Key (CIK) is 0000320193. As of May 22 2025, Apple's market capitalization is 3158180407700 United States dollars, and the company has 164000 employees. Apple's homepage URL is https://www.apple.com.


In [52]:
send_message("Tell me about Amazon's trends")

#### Natural language response

As of May 2025, Amazon has the following analyst recommendation trends:

*   May 1, 2025: 51 buy, 6 hold, 0 sell, 22 strongBuy, 0 strongSell
*   April 1, 2025: 50 buy, 4 hold, 0 sell, 23 strongBuy, 0 strongSell
*   March 1, 2025: 51 buy, 5 hold, 0 sell, 21 strongBuy, 0 strongSell
*   February 1, 2025: 52 buy, 5 hold, 0 sell, 21 strongBuy, 0 strongSell

In [55]:
send_message("What is Google's stock ticker symbol?")

#### Natural language response

Google's stock ticker symbols are GOOGL and GOOG on the NASDAQ. It is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1. These symbols now refer to Alphabet Inc., Google's holding company.


In [58]:
send_message("What is MGM Studio's stock symbol?")

#### Natural language response

I was unable to find the stock symbol for Mgm Studios.

In [60]:
send_message("What is Amazon MGM Studio's stock symbol?")

#### Natural language response

Amazon's stock symbol is AMZN and it is listed on the NASDAQ.


In [63]:
send_message("What is Facebook's stock ticker symbol?")

Score similarity to stored grounding: 0it [00:00, ?it/s]


#### Natural language response

Facebook's stock ticker symbol is META. It was formerly FB until June 9, 2022.


In [ ]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

In [ ]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

In [ ]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

In [ ]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.
    Avoid discussing duplicate news entries.''')

# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>